# Clinical Code Toolbox
### Week 4 Exercise — Stella Oiro (Andela AI Engineering Bootcamp)

A two-tab AI-powered toolbox for clinical Python code.

**Tab 1 — Clinical Docstring Generator**
Paste a clinical calculation function and get back medical-grade docstrings:
units, normal ranges, clinical references, and parameter constraints.

**Tab 2 — Clinical Unit Test Generator**
Paste a clinical function and get back pytest unit tests covering:
typical values, edge cases (paediatric, renal impairment, obesity),
and safety boundaries (overdose thresholds, critical lab values).

**Models:** GPT-4.1-mini (OpenAI) or Llama 3.2 (Ollama, local)

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

In [ ]:
# ── Clients ───────────────────────────────────────────────────────────────────

openai_client = OpenAI()
ollama_client = OpenAI(api_key="ollama", base_url="http://localhost:11434/v1")

GPT_MODEL   = "gpt-4.1-mini"
LLAMA_MODEL = "llama3.2"

models  = [GPT_MODEL, LLAMA_MODEL]
clients = {GPT_MODEL: openai_client, LLAMA_MODEL: ollama_client}

print("Clients ready.")

In [ ]:
# ── System prompts ────────────────────────────────────────────────────────────

DOCSTRING_PROMPT = """\
You are a senior clinical software engineer with expertise in medical informatics.
Your task is to read the provided Python function and return it with a comprehensive,
PEP-257-compliant docstring and clear inline comments.

The docstring must include:
- A concise one-line summary of the clinical purpose
- A longer description explaining the underlying clinical formula or guideline
- Args section: each parameter with type, units (e.g. mg/kg, mmol/L), valid range, and clinical meaning
- Returns section: type, units, and clinical interpretation
- Raises section: any ValueError or clinical safety constraints
- A clinical reference (e.g. NICE guideline, WHO formula, published equation)
- At least one Example showing typical clinical use

Inline comments should explain clinical reasoning, not restate the code.
Do not modify the function logic or variable names.
Output only the annotated function — no extra text.
"""

UNIT_TEST_PROMPT = """\
You are a senior clinical software engineer with expertise in medical informatics and software testing.
Your task is to read the provided Python function and generate a comprehensive pytest test suite.

Tests must cover:
- Typical adult values (normal clinical range)
- Boundary values at the edge of normal ranges
- Paediatric cases (if weight or age is a parameter)
- Obesity or extreme body habitus
- Renal impairment or hepatic impairment (if relevant)
- Critical / safety-threshold values (e.g. toxic drug levels, critical lab values)
- Invalid inputs that should raise ValueError
- Zero or negative inputs where clinically impossible

Use descriptive test names (test_<function>_<scenario>).
Add a brief comment above each test explaining the clinical scenario.
Use pytest.approx for floating-point comparisons.
Output only the test code — no extra text.
"""

print("System prompts defined.")

In [ ]:
# ── Example clinical functions ────────────────────────────────────────────────
# Pre-loaded in the UI so users can try immediately without writing code.

EXAMPLE_BMI = """\
def calculate_bmi(weight_kg, height_m):
    if weight_kg <= 0 or height_m <= 0:
        raise ValueError("Weight and height must be positive.")
    bmi = weight_kg / (height_m ** 2)
    if bmi < 18.5:
        category = "Underweight"
    elif bmi < 25.0:
        category = "Normal weight"
    elif bmi < 30.0:
        category = "Overweight"
    else:
        category = "Obese"
    return round(bmi, 1), category
"""

EXAMPLE_EGFR = """\
def calculate_egfr_ckd_epi(creatinine_umol_L, age, sex):
    creatinine_mg_dL = creatinine_umol_L / 88.42
    if sex.lower() == "female":
        kappa, alpha, mult = 0.7, -0.241, 1.012
    else:
        kappa, alpha, mult = 0.9, -0.302, 1.0
    ratio = creatinine_mg_dL / kappa
    if ratio < 1:
        egfr = 142 * (ratio ** alpha) * (0.9938 ** age) * mult
    else:
        egfr = 142 * (ratio ** -1.200) * (0.9938 ** age) * mult
    return round(egfr, 1)
"""

EXAMPLE_DRUG_DOSE = """\
def calculate_amoxicillin_dose(weight_kg, age_years, indication):
    if weight_kg <= 0 or age_years < 0:
        raise ValueError("Weight must be positive and age must be non-negative.")
    standard_doses = {
        "otitis_media": 40,
        "pneumonia": 90,
        "strep_throat": 50,
    }
    if indication not in standard_doses:
        raise ValueError(f"Unknown indication: {indication}")
    dose_per_kg = standard_doses[indication]
    total_dose_mg = weight_kg * dose_per_kg
    max_dose_mg = 500 if age_years >= 12 else 250
    return min(total_dose_mg, max_dose_mg)
"""

print("Example functions ready.")

In [ ]:
# ── Generation functions (streaming) ─────────────────────────────────────────

def generate_docstring(code, model):
    """Stream a clinical docstring for the given function."""
    stream = clients[model].chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": DOCSTRING_PROMPT},
            {"role": "user",   "content": code}
        ],
        stream=True
    )
    output = ""
    for chunk in stream:
        output += chunk.choices[0].delta.content or ""
        yield output.replace("```python", "").replace("```", "")


def generate_tests(code, model):
    """Stream clinical pytest unit tests for the given function."""
    stream = clients[model].chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": UNIT_TEST_PROMPT},
            {"role": "user",   "content": code}
        ],
        stream=True
    )
    output = ""
    for chunk in stream:
        output += chunk.choices[0].delta.content or ""
        yield output.replace("```python", "").replace("```", "")


print("Generation functions ready.")

In [ ]:
# ── Gradio UI ─────────────────────────────────────────────────────────────────

with gr.Blocks(title="Clinical Code Toolbox", theme=gr.themes.Soft()) as ui:

    gr.Markdown("""
    # Clinical Code Toolbox
    AI-powered docstrings and unit tests for clinical Python functions.
    Paste any clinical calculation below — BMI, eGFR, drug dosing, lab interpretation.
    """)

    with gr.Tab("Docstring Generator") as tab_doc:
        gr.Markdown("### Paste a clinical function to generate a medical-grade docstring and inline comments.")
        with gr.Row():
            with gr.Column():
                code_in_doc = gr.Code(
                    label="Clinical function",
                    language="python",
                    lines=22,
                    value=EXAMPLE_BMI
                )
                model_dd_doc = gr.Dropdown(
                    choices=models,
                    value=GPT_MODEL,
                    label="Model"
                )
                gr.Examples(
                    examples=[
                        [EXAMPLE_BMI,       GPT_MODEL],
                        [EXAMPLE_EGFR,      GPT_MODEL],
                        [EXAMPLE_DRUG_DOSE, GPT_MODEL],
                    ],
                    inputs=[code_in_doc, model_dd_doc],
                    label="Example functions"
                )
                doc_btn = gr.Button("Generate Docstring", variant="primary")
            with gr.Column():
                doc_out = gr.Code(
                    label="Annotated function",
                    language="python",
                    lines=22
                )

        doc_btn.click(
            fn=generate_docstring,
            inputs=[code_in_doc, model_dd_doc],
            outputs=doc_out
        )

    with gr.Tab("Unit Test Generator") as tab_test:
        gr.Markdown("### Paste a clinical function to generate pytest tests covering clinical edge cases.")
        with gr.Row():
            with gr.Column():
                code_in_test = gr.Code(
                    label="Clinical function",
                    language="python",
                    lines=22,
                    value=EXAMPLE_BMI
                )
                model_dd_test = gr.Dropdown(
                    choices=models,
                    value=GPT_MODEL,
                    label="Model"
                )
                gr.Examples(
                    examples=[
                        [EXAMPLE_BMI,       GPT_MODEL],
                        [EXAMPLE_EGFR,      GPT_MODEL],
                        [EXAMPLE_DRUG_DOSE, GPT_MODEL],
                    ],
                    inputs=[code_in_test, model_dd_test],
                    label="Example functions"
                )
                test_btn = gr.Button("Generate Unit Tests", variant="primary")
            with gr.Column():
                test_out = gr.Code(
                    label="Generated tests",
                    language="python",
                    lines=22
                )

        test_btn.click(
            fn=generate_tests,
            inputs=[code_in_test, model_dd_test],
            outputs=test_out
        )

    # Sync code between tabs when switching
    tab_doc.select(lambda c: c,  inputs=code_in_test, outputs=code_in_doc)
    tab_test.select(lambda c: c, inputs=code_in_doc,  outputs=code_in_test)

print("UI built.")

In [ ]:
ui.launch(inbrowser=True)